# Detección de barras en galaxias - Modelo "Baseline"
## Proyecto integrador MNA

### Integrantes
- Jonathan Jesús Marmolejo Hernández - A01795195
- Isaid Posadas Oropeza - A01795015
- Luis Daniel Ortega Muñoz - A01795197

In [ ]:
# Uncomment this if running in Google Colab. It will install the bargal package from GitHub.
# !pip install git+https://github.com/ludanortmun/itesm-mna-barred-galaxies.git

## Importando librerías

In [1]:
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from PIL import Image

from bargal.dataset.load import load_dataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


## Preparando el conjunto de datos

El primer paso consiste en cargar nuestro conjunto de datos y dividirlo en conjunto de entrenamiento, validación y prueba.

In [6]:
dataset_path = '../data/dataset.csv'

df = load_dataset(dataset_path)

df.info()

# TODO: Do an actual train-valid-test split
train_df = df.iloc[:200]
valid_df = df.iloc[200:250]
test_df = df.iloc[250:300]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10126 entries, 0 to 10125
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    10126 non-null  object 
 1   objra   10126 non-null  float64
 2   objdec  10126 non-null  float64
 3   Bars    10126 non-null  float64
dtypes: float64(3), object(1)
memory usage: 316.6+ KB


Ahora creamos nuestra clase para representar el conjunto de datos, heredando de Dataset de PyTorch. El dataset a utilizar para nuestra red neuronal consiste en imágenes de galaxias y su respectiva etiqueta, que indica si la galaxia tiene o no una barra.

La etiqueta es un tensor de tamaño 1, donde 1 indica que la galaxia tiene una barra y 0 indica que no. Sin embargo, el conjunto de datos original no tiene etiquetas binarias, sino múltiples categorías indicando el tipo de barra que tiene la galaxia. Por lo tanto, convertimos cualquier etiqueta que represente la presencia de una barra (independientemente de sus características) a 1 y cualquier etiqueta que represente la ausencia de una barra a 0; de forma similar a como se realizó en el entregable [Avance1.Equipo22.ipynb](https://github.com/ludanortmun/itesm-mna-barred-galaxies/tree/main/notebooks/Avance1.Equipo22.ipynb), donde se creó la columna `has_bar` derivada de `Bars`.

En cuanto a la carga de imágenes, este conjunto de datos consiste en las imágenes de galaxias previamente procesadas en formato PNG. El preprocesamiento consiste, principalmente, en la sustracción de las bandas G y R para enfatizar las estructuras de barras. Las imágenes resultantes tienen dimensiones de 400x400 píxeles y están en escala de grises. Estas imágenes son cargadas utilizando la librería PIL y convertidas a tensores.

El script de preprocesamiento puede ser consultado en este enlace: [bargal/commands/preprocess.py](https://github.com/ludanortmun/itesm-mna-barred-galaxies/blob/297f69b278ea6bc5099ef23a0d539602995bc55e/bargal/commands/preprocess.py)

El conjunto de imágenes pre procesadas puede descargarse con el siguiente enlace: [dataset.processed.GRLogDiff](https://tecmx-my.sharepoint.com/:u:/g/personal/a01795197_tec_mx/EexaLnqaLLdCt1JNxLib8VYBeOHJo95vuOr-Pfxv-55Iww?e=0gfeuq)


In [12]:
class GalaxiesDataset(Dataset):
    def __init__(self, galaxies_df: pd.DataFrame, img_dir: str):
        self.labels = []
        self.images = []

        for i in range(len(galaxies_df)):
            label = 1 if galaxies_df.iloc[i]['Bars'] != 0 else 0
            galaxy_name = galaxies_df.iloc[i]['name']
            img = Image.open(f"{img_dir}/{galaxy_name}.png")

            self.labels.append(torch.tensor(label).to(device).float())
            self.images.append(transforms.ToTensor()(img).to(device))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [13]:
n = 32
processed_images_path = '../data/processed'

train_data = GalaxiesDataset(train_df, processed_images_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

valid_data = GalaxiesDataset(valid_df, processed_images_path)
valid_loader = DataLoader(valid_data, batch_size=n)
valid_N = len(valid_loader.dataset)

## Definiendo el modelo

Debido a que estamos trabajando con imágenes, utilizaremos una red neuronal convolucional (CNN) como modelo base. La arquitectura de la red es la siguiente:

In [16]:
# TODO: define the model architecture

model = nn.Sequential(
    # First conv layer
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),  # 32 x 400 x 400
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2), # 32 x 200 x 200

    ## Flattening
    nn.Flatten(),
    nn.Linear(400*400, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, 2)
)

In [17]:
model = torch.compile(model.to(device))
model

OptimizedModule(
  (_orig_mod): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=160000, out_features=512, bias=True)
    (6): Dropout(p=0.3, inplace=False)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=2, bias=True)
  )
)

### Función de pérdida y optimizador

Debido a que estamos trabajando con un problema de clasificación binaria, utilizaremos la función de pérdida `BCEWithLogitsLoss`. El optimizador a utilizar es Adam.

In [18]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters())
my_model = model.to(device)

## Entrenamiento

### Definiendo la función de entrenamiento

In [ ]:
def get_batch_accuracy(output, y, N):
    zero_tensor = torch.tensor([0]).to(device)
    pred = torch.gt(output, zero_tensor)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [ ]:
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print(f'Train - Loss: {loss:.4f} Accuracy: {accuracy:.4f}')

In [ ]:
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print(f'Valid - Loss: {loss:.4f} Accuracy: {accuracy:.4f}')

### Ejecución del entrenamiento

In [ ]:
epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

### Guardando el modelo

In [ ]:
model_path = '../models/model.pth'
torch.save(model.state_dict(), model_path)

## Evaluación del modelo

Una vez entrenada la red neuronal, procedemos a evaluar su desempeño en el conjunto de prueba.

In [ ]:
# TODO: Using the train dataset, compute classification report, confusion matrix, etc.